## 데이터 로드

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import pandas as pd
import numpy as np

#! pip install sktime
from sktime.forecasting.model_selection import temporal_train_test_split

In [15]:
data_file = "C:/Users/yonghwa/Desktop/4-2/인공지능 금융 통계/AIFT/data.csv"
df = pd.read_csv(data_file,index_col='Unnamed: 0') #
df = df.iloc[1:][:]
print(df.shape)
df.head()

(14201, 27)


,ma_w_x,ma20_w_x,ma60_w_x,macd_w_x,macdsignal_w_x,macdhist_w_x,rsi_w_x,ad_w_x,ts_end_x,ts_start_x,...,macdsignal_w_y,macdhist_w_y,rsi_w_y,ad_w_y,ts_end_y,ts_start_y,is_higher_y,offset_intra_day_y,target_y,label
2021-12-02 11:12:00,32.906667,78.280,29.598321,8.622361,22.457295,-13.834934,-1.847265,51515.668116,False,False,...,-2.542392,1.449403,1.920541,-541806.0,False,False,False,0.330769,0.998628,NOP
2021-12-02 11:15:00,35.760000,74.620,28.295216,2.547207,18.475277,-15.928070,-5.503149,25495.709956,False,False,...,-2.110002,1.729559,7.159809,-309922.4,False,False,False,0.338462,0.997340,X
2021-12-02 11:18:00,42.043333,71.785,27.083512,-2.153807,14.349460,-16.503267,-4.424460,21430.047899,False,False,...,-1.723365,1.546547,2.957516,-377094.0,False,False,False,0.346154,0.998507,X
2021-12-02 11:21:00,46.373333,71.240,25.660619,-6.761681,10.127232,-16.888913,-4.671662,15676.338589,False,False,...,-1.278042,1.781292,8.215010,-245900.2,False,False,False,0.353846,0.997179,X
2021-12-02 11:24:00,53.736667,70.095,25.342989,-6.298201,6.842146,-13.140346,0.011823,24676.029280,False,False,...,-0.888162,1.559519,4.020271,-663077.5,False,False,False,0.361538,0.998305,NOP


In [16]:
effective_cols = [

       'ma_w_x', 'ma20_w_x', 'ma60_w_x', 'macd_w_x', 'macdsignal_w_x', 'macdhist_w_x', 'rsi_w_x', 'ad_w_x', 'is_higher_x', 
       'ts_end_x', 'ts_start_x', 'offset_intra_day_x', 
       'ma_w_y',  'ma20_w_y', 'ma60_w_y','macd_w_y', 'macdsignal_w_y', 'macdhist_w_y', 'rsi_w_y', 'ad_w_y', 'is_higher_y', 


       'label'
       
       ]

In [17]:
data_df = df[effective_cols].dropna(axis=0)

## 데이터 셋 분할, 전처리

In [18]:
learn_split = data_df.loc[:'2022-09-30']
test_split = data_df.loc['2022-10-01':'2022-12-08']

In [19]:
X_train, X_val = temporal_train_test_split(learn_split, test_size=0.2)
X_test = test_split

In [20]:
y_train = X_train.pop('label')
y_val = X_val.pop('label')
y_test = X_test.pop('label')

In [9]:
y_train.value_counts(normalize=True)

NOP    0.910646
Y      0.048898
X      0.040456
Name: label, dtype: float64

In [10]:
y_val.value_counts(normalize=True)

NOP    0.943446
X      0.029811
Y      0.026743
Name: label, dtype: float64

In [11]:
y_test.value_counts(normalize=True)

NOP    0.912655
X      0.055512
Y      0.031832
Name: label, dtype: float64

In [30]:
# ts_end_x, ts_start_x
data_df.groupby('ts_end_x').count()

,ma_w_x,macd_w_x,macdsignal_w_x,macdhist_w_x,rsi_w_x,ad_w_x,is_higher_x,ts_start_x,offset_intra_day_x,ma_w_y,macd_w_y,macdsignal_w_y,macdhist_w_y,rsi_w_y,ad_w_y,is_higher_y,label
ts_end_x,,,,,,,,,,,,,,,,,
False,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145


In [31]:
data_df.groupby('ts_start_x').count()

,ma_w_x,macd_w_x,macdsignal_w_x,macdhist_w_x,rsi_w_x,ad_w_x,is_higher_x,ts_end_x,offset_intra_day_x,ma_w_y,macd_w_y,macdsignal_w_y,macdhist_w_y,rsi_w_y,ad_w_y,is_higher_y,label
ts_start_x,,,,,,,,,,,,,,,,,
False,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145


In [21]:
X_train.drop(columns=['ts_end_x','ts_start_x'],inplace=True)
X_val.drop(columns=['ts_end_x','ts_start_x'],inplace=True)
X_test.drop(columns=['ts_end_x','ts_start_x'],inplace=True)

X_train = X_train.astype({'is_higher_x':'bool','is_higher_y':'bool'})
X_val = X_val.astype({'is_higher_x':'bool','is_higher_y':'bool'})
X_test = X_test.astype({'is_higher_x':'bool','is_higher_y':'bool'})

C:\Users\yonghwa\AppData\Local\Temp\ipykernel_1780\1501966215.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns=['ts_end_x','ts_start_x'],inplace=True)


ts_end_x, ts_start_x 값이 전부 False로 

In [41]:
# imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_resampled, y_resampled = smt.fit_resample(X_train, y_train)

## labelencoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_resampled)
y_labelencoder = label_encoder.transform(y_resampled)
# NOP = 0 , X = 1, Y = 2

from sklearn.model_selection import train_test_split
X_resampled_train, X_resampled_val, y_resampled_train, y_resampled_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [42]:
print(y_resampled)
print(y_labelencoder)

0        NOP
1          X
2          X
3        NOP
4        NOP
        ... 
17970      Y
17971      Y
17972      Y
17973      Y
17974      Y
Name: label, Length: 17975, dtype: object
[0 1 1 ... 2 2 2]


In [33]:
y_resampled_train.value_counts(normalize=True)

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [15]:
y_resampled_val.value_counts(normalize=True)

X      0.348223
Y      0.330215
NOP    0.321562
Name: label, dtype: float64

## flaml으로 최적 Hyperparameter 가져오기

In [38]:
# !pip install flaml
from flaml import AutoML
automl = AutoML()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 208 kB 7.3 MB/s 
     |████████████████████████████████| 2.0 MB 60.7 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [40]:
automl.fit(X_resampled_train, y_resampled_train, 
          task="classification", 
          X_val=X_resampled_val, y_val=y_resampled_val, 
          time_budget=60*10, 
          log_file_name='.segment_automl_baseline2_10m.log', 
          estimator_list=["xgboost"])

[flaml.automl: 12-11 17:25:54] {2599} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 12-11 17:25:54] {2601} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 12-11 17:25:54] {2604} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 12-11 17:25:54] {2726} INFO - Minimizing error metric: log_loss
INFO:flaml.automl:Minimizing error metric: log_loss
[flaml.automl: 12-11 17:25:54] {2870} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 12-11 17:25:54] {3166} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 12-11 17:25:54] {3296} INFO - Estimated sufficient time budget=2558s. Estimated necessary time budget=3s.
INFO:flaml.automl:Estimated sufficient time budget=2558s. Estimated necessary time budget=3s.
[flaml.automl

## Model 적합 & 결과

In [29]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
model = XGBClassifier(colsample_bylevel=0.7845109195865365,
                      colsample_bytree=0.6447843507910973,
                      grow_policy='lossguide',
                      learning_rate=0.2856559413650925,
                      max_depth=0,
                      max_leaves=203,
                      min_child_weight=0.3269890996192655,
                      n_estimators=696,
                      n_jobs=-1,
                      objective='multi:softprob',
                      reg_alpha=0.03134269983049553,
                      reg_lambda=0.004570510428599539,
                      subsample=1.0,
                      tree_method='hist',
                      use_label_encoder=False,
                      verbosity=0
                      )

In [30]:
y_resampled_train.value_counts()

NOP  X  Y
0    0  1    4913
     1  0    4876
1    0  0    4591
dtype: int64

In [34]:
model.fit(X_resampled_train, y_resampled_train, early_stopping_rounds=200,
           eval_metric='merror', eval_set=[(X_resampled_val, y_resampled_val)])

[0]	validation_0-merror:0.56912
[1]	validation_0-merror:0.56134
[2]	validation_0-merror:0.58526
[3]	validation_0-merror:0.56885
[4]	validation_0-merror:0.56412
[5]	validation_0-merror:0.55577


c:\Anaconda3\envs\aift\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[6]	validation_0-merror:0.55327
[7]	validation_0-merror:0.54604
[8]	validation_0-merror:0.52768
[9]	validation_0-merror:0.52072
[10]	validation_0-merror:0.51738
[11]	validation_0-merror:0.53213
[12]	validation_0-merror:0.53741
[13]	validation_0-merror:0.53714
[14]	validation_0-merror:0.52935
[15]	validation_0-merror:0.52629
[16]	validation_0-merror:0.52629
[17]	validation_0-merror:0.51933
[18]	validation_0-merror:0.51460
[19]	validation_0-merror:0.51738
[20]	validation_0-merror:0.50960
[21]	validation_0-merror:0.50264
[22]	validation_0-merror:0.50654
[23]	validation_0-merror:0.50125
[24]	validation_0-merror:0.49346
[25]	validation_0-merror:0.50515
[26]	validation_0-merror:0.49764
[27]	validation_0-merror:0.50097
[28]	validation_0-merror:0.49402
[29]	validation_0-merror:0.50097
[30]	validation_0-merror:0.49458
[31]	validation_0-merror:0.49152
[32]	validation_0-merror:0.48540
[33]	validation_0-merror:0.48734
[34]	validation_0-merror:0.49235
[35]	validation_0-merror:0.49430
[36]	validatio

XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.7845109195865365, colsample_bynode=1,
              colsample_bytree=0.6447843507910973, enable_categorical=False,
              gamma=0, gpu_id=-1, grow_policy='lossguide', importance_type=None,
              interaction_constraints='', learning_rate=0.2856559413650925,
              max_delta_step=0, max_depth=0, max_leaves=203,
              min_child_weight=0.3269890996192655, missing=nan,
              monotone_constraints='()', n_estimators=696, n_jobs=-1,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0.03134269983049553,
              reg_lambda=0.004570510428599539, scale_pos_weight=None,
              subsample=1.0, tree_method='hist', use_label_encoder=False, ...)

In [35]:
'''compute predictions of testing dataset''' 
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

In [36]:
pd.Series(y_pred).unique()

array([0, 2, 1])

In [37]:
pd.Series(y_pred).value_counts(normalize=True)

0    0.600543
2    0.222826
1    0.176630
dtype: float64

In [46]:
y_test.value_counts(normalize=True)

NOP    0.912655
X      0.055512
Y      0.031832
Name: label, dtype: float64

In [47]:
# confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
 
cm = confusion_matrix(y_test,y_pred)
cm

array([[2085,  121,  145],
       [ 132,    4,    7],
       [  70,    1,   11]])

In [48]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         NOP       0.91      0.89      0.90      2351
           X       0.03      0.03      0.03       143
           Y       0.07      0.13      0.09        82

    accuracy                           0.82      2576
   macro avg       0.34      0.35      0.34      2576
weighted avg       0.84      0.82      0.83      2576



In [46]:
# 모델 저장
'''pickle and save the automl object'''
import pickle
with open('model_xgboost.pkl', 'wb') as f:
    pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)
'''load pickled automl object'''
with open('model_xgboost.pkl', 'rb') as f:
    model = pickle.load(f)